# Persistent Data Structures

#### Goncalo Pinto - fc58178


<div align="center">
<a href="#introduction"><kbd> <br>Introduction<br> </kbd></a>&ensp;&ensp;
<a href="#motivation"><kbd> <br>Motivation & historical background<br> </kbd></a>&ensp;&ensp;
<a href="#design"><kbd> <br>Design of the algorithms<br> </kbd></a>&ensp;&ensp;
<a href="#reference"><kbd> <br>References<br> </kbd></a>&ensp;&ensp;
</div>


<a id="introduction"></a>
<img src="https://readme-typing-svg.herokuapp.com?font=Lexend+Giga&size=25&pause=1000&color=CCA9DD&vCenter=true&width=350&height=25&lines=Introduction" width="600"/>

A persistent data structure is a data structure that preserves the previous versions of itself when modified, allowing access to any historical version. In other words, once a change is made to the structure, both the original and modified versions remain accessible. This is particularly useful in scenarios where you need to keep track of the history of updates or backtrack to previous states of the data structure.

- A data structure is `partially persistent` if all versions can be accessed but only the newest version can be modified. 

- `Fully persistent` if every version can be both accessed and modified. 

- `Confluently persistent` is when we merge two or more versions to get a new version.

These types of data structures are particularly common in logical and functional programming, as languages in those paradigms discourage (or fully forbid) the use of mutable data. 


## Partially persistent data structure

A partially persistent data structure is a data structure in which old versions are remembered and can always be inspected. However, only the latest version of the data structure can be modified.

General theoretical schemes are known (e.g. the fat node method ) for making any data structure partially persistent.

#### Pros

- You can query any previous version of the data structure
- Simpler to implement and more efficient than fully or confluently persistent structures.
- Number of versions and extra memory needed is often limited.
- Worst-case guarantees, updates and queries are predictable.

#### Cons

- You can’t modify old versions — only the latest one can be updated.
- New version created on every update, which can increase memory usage over time.
- Requires extra logic for version tracking. Compared to regular (non-persistent) data structures, persistence adds code and logic complexity.

## Fully persistent data structure

A fully persistent structure offers accesses to its previous versions for queries and updates, where each update operation on a version of the data structure creates a new branch from this version for the new version.

In a fully persistent structure, if update operation i applies to version j < i, the result of the update is version i; version j is not changed by the update. We denote by n the number of nodes in the current version.

#### Pros

- You can update any version, not just the latest — each update creates a new branch in the version tree.
- Ideal for situations where different branches of computation must operate independently from the same data snapshot.
- Old versions remain unchanged, ensuring reproducibility and safety.

#### Cons

- More difficult to implement than partial persistence
- Branching versions can lead to faster memory growth, especially if changes are frequent or deep.
- Multiple active branches may make the logic and debugging of versioned behavior more complex.
- Some operations on older versions may become slower (e.g., in ropes or trees) due to versioning overhead.

## Confluently persistent data structure

A data structure is called confluently persistent if there is a meld operation that creates a new version from two previous versions so that branches in a version tree are joined and a version DAG is formed.

Confluently Persistent Sets and Maps are functional binary search trees that support efficient set operations both when operands are disjoint and when they are overlapping.

#### Pros

- Supports a meld operation: you can combine two or more past versions to create a new version
- Unlike full persistence (which forms a tree), confluent persistence allows merging branches into a directed acyclic graph (DAG) of versions.
- Ideal for use cases involving merging divergent states (distributed systems, version control...)
- Excellent for multi-user environments or simulations that need to branch and later reconcile state.

#### Cons

- Requires careful tracking of merged nodes and resolution of conflicts — significant algorithmic overhead.
- If two merged versions changed the same data, you need a way to resolve which value wins.
- Tracking and combining overlapping histories can result in significant duplication.
- Ensuring structural invariants hold after merges adds implementation complexity.

## Partial versus full persistence

In the partial persistence model, a programmer may query any previous version of a data structure, but may only update the latest version. This implies a linear ordering among each version of the data structure. 

In the fully persistent model, both updates and queries are allowed on any version of the data structure. In some cases the performance characteristics of querying or updating older versions of a data structure may be allowed to degrade, as is true with the rope data structure. More flexible than partial persistence — you can branch off from any point in the past and continue.

<a id="motivation"></a>

---

<a id="motivation&historicalbackground"></a>
<img src="https://readme-typing-svg.herokuapp.com?font=Lexend+Giga&size=25&pause=1000&color=CCA9DD&vCenter=true&width=350&height=25&lines=Motivation &" width="600"/>

<img src="https://readme-typing-svg.herokuapp.com?font=Lexend+Giga&size=25&pause=1000&color=CCA9DD&vCenter=true&width=350&height=25&lines=Background" width="600"/>

The concept of persistent data structure was introduced in Driscoll, Sarnak, Sleator, and Tarjan's 1986 article "Making Data Structures Persistent". 

The paper explores the advantages of using a data structure powerfull enghou to store previous stages of it self and compares it to a ephemeral one. It also exposed the 3 different types of persistence: Partial Persistence, Full Persistence and Confluent Persistence.

The study intruduced different thecniques such as: The Fat Node Method, The Node-Copying Method or the The Node-Splitting Method.

Over the years, there has been significant progress in this area, leading to more efficient implementations of persistent data structures.

The paper titled "Partially Persistent Data Structures of Bounded Degree with Constant Update Time" by Gerth Stølting Brodal (1996) presents a method for making data structures partially persistent while guaranteeing constant worst-case time for updates and access operations — a significant improvement over earlier techniques that only offered amortized efficiency.



---

<a id="design"></a>
<img src="https://readme-typing-svg.herokuapp.com?font=Lexend+Giga&size=25&pause=1000&color=CCA9DD&vCenter=true&width=350&height=25&lines=Algorithms Design" width="600"/>


---

<a id="references"></a>
<img src="https://readme-typing-svg.herokuapp.com?font=Lexend+Giga&size=25&pause=1000&color=CCA9DD&vCenter=true&width=350&height=25&lines=References" width="600"/>


### Articles and Tutorials
- [Introduction to Persistent Data Structures](https://arpitbhayani.me/blogs/persistent-data-structures-introduction/) - A beginner-friendly overview of persistent data structures.
- [Partial Persistence](https://sungsoo.github.io/2014/01/18/partial-persistence.html) - A concise explanation of partial persistence.

### Research Papers
- [Partially Persistent Data Structures of Bounded Degree with Constant Update Time](https://www.cs.au.dk/~gerth/papers/njc96.pdf) - Gerth Stølting Brodal's seminal paper on partially persistent data structures.
- [MIT Advanced Algorithms Lecture Notes](https://ocw.mit.edu/courses/6-854j-advanced-algorithms-fall-2005/2165d83010dc7633bce397ea75f889f9_lec05_1999.pdf) - Lecture notes discussing persistence in data structures.
- https://www.cs.cmu.edu/~sleator/papers/making-data-structures-persistent.pdf
- https://arxiv.org/pdf/1301.3388
- https://www.cs.cmu.edu/~sleator/papers/another-persistence.pdf

### Online Guides
- [USACO Guide: Persistent Data Structures](https://usaco.guide/adv/persistent?lang=cpp) - A practical guide for competitive programming.

### Encyclopedic Resources
- [Wikipedia: Persistent Data Structure](https://en.wikipedia.org/wiki/Persistent_data_structure) - A general overview of persistent data structures.

### Additional Resources
- [Lirias Repository](https://lirias.kuleuven.be/retrieve/19369) - A collection of academic resources on persistence.
